# data preprocessing (데이터 전처리)

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import urllib.request
import webbrowser
import pandas as pd
import requests
from html_table_parser import parser_functions as parser
import time


##### 데이타 데이스 #####
import pymysql
import mysql.connector

# Score_Table ( list, dataframe )

In [2]:
########## SCORE table 내용 스크랩 ###########
##### 순위 페이지 랭킹(스코어)테이블 #####
res = requests.get('https://flixpatrol.com/top10/netflix/world/2021/full/#netflix-2')
soup = BeautifulSoup(res.content, 'xml')
 
netfl2 = soup.find('div',{'id':'netflix-2'})
table1 = netfl2.find('tbody',{'class':'tabular-nums'})
html_table_21_sub = parser.make2d(table1)
#html_table1

##### 2020년 ####
res_20 = requests.get('https://flixpatrol.com/top10/netflix/world/2020/full/#netflix-2')
soup_20 = BeautifulSoup(res_20.content, 'xml')

netfl2_20 = soup_20.find('div',{'id':'netflix-2'})
table1_20 = netfl2_20.find('tbody',{'class':'tabular-nums'})
html_table_20_sub = parser.make2d(table1_20)

In [3]:
##### replace()이용하여 문자열의 쉼표 (,) 제거하자 #####
html_table_21 = []
html_table_20 = []

for i in range(len(html_table_21_sub)):
    temp_list=[]
    for j in range(len(html_table_21_sub[i])):
        temp21 = html_table_21_sub[i][j].replace(",","")
        if temp21 != '':
            temp_list.append(temp21)
        else :
            temp_list.append('0')
        #temp_list.append(temp21)
    html_table_21.append(temp_list)

    
for i in range(len(html_table_20_sub)):
    temp_list=[]
    for j in range(len(html_table_20_sub[i])):
        temp20 = html_table_20_sub[i][j].replace(",","")
        if temp20 != '':
            temp_list.append(temp20)
        else :
            temp_list.append('0')
    html_table_20.append(temp_list)


In [10]:
##### 데이터 프레임으로 만들기#####

#### 2021 데이터프레임 ####
df_main_21 = pd.DataFrame(html_table_21)
df_main_21 = df_main_21.drop(columns=[0,1,3,4])
df_main_21 = df_main_21.drop_duplicates(ignore_index=True)
df_main_21.columns=["title","num_c","avg_c","days","avg_day","total_score"]
df_main_21["year"]="2021"
df_main_21 = df_main_21[df_main_21['total_score']!="0"]

#### 2020 데이터프레임####
df_main_20 = pd.DataFrame(html_table_20)
df_main_20 = df_main_20.drop(columns=[0,1,3,4])
df_main_20 = df_main_20.drop_duplicates(ignore_index=True)
df_main_20.columns=["title","num_c","avg_c","days","avg_day","total_score"]
df_main_20["year"]="2020"
df_main_20 = df_main_20[df_main_20['total_score']!="0"]


# Detail_Table ( list, dataframe )

In [5]:
############# Detail table #############
##### 각 상세페이지의 url 가져오기 #####

html = requests.get("https://flixpatrol.com/top10/netflix/world/2021/full/#netflix-2").text
soup = BeautifulSoup(html,"lxml")

html_2020 = requests.get("https://flixpatrol.com/top10/netflix/world/2020/full/#netflix-2").text
soup_2020 = BeautifulSoup(html_2020,"lxml")

#### 주소받아올 리스트 ####
url=[] 

##### 2021 년도 데이터  url에 추가 #####
for href in soup.find("div",{"id":"netflix-2"}).find_all("td"):
    try:
        href.find("a")["href"]
        url.append("https://flixpatrol.com" + href.find("a")["href"])
    except:
        pass
    
##### 2020 년도 데이터  url에 추가 #####
for href in soup_2020.find("div",{"id":"netflix-2"}).find_all("td"):
    try:
        href.find("a")["href"]
        url.append("https://flixpatrol.com" + href.find("a")["href"])
    except:
        pass

m_list = url
url_re = []
for i in m_list:
    if i not in url_re:
        url_re.append(i)

#print(len(new_url))

In [45]:
###################### Detail table 내용 스크랩 ##########################
##### 상세페이지의 정보 가져오기 (제목 분류 국가 제작사 장르 키워드) #####
##### //////////       ※주의※ 오 래 걸 림 ! ! ! ! !!  /////////////#####

list_all = [] 

for i in range(len(url_re)):
    html_2 = urllib.request.urlopen(url_re[i])
    soup_2 = BeautifulSoup(html_2,"html.parser")
    info_title = soup_2.find("h1",{"class":"mb-3"})
    info_d= soup_2.find("div",{"class":"flex flex-wrap text-sm leading-6 text-gray-500"})
    spans = info_d.find_all("span")

    ready1=[]

    
    #####리스트에 제목넣음#####
    ready1.append(info_title.get_text())
    
    ##### 1초 ? 쉬어가기 zzzZ #####
    time.sleep(1)    

    #####리스트 1행 씩 생성#####
    for t in range(0,len(spans)):
        excp = spans[t].get_text()
        if excp != "|" and excp != "\n\n" and ("/" in excp)==False:
            ready1.append(spans[t].get_text())   

    #####문자열 replace, \n 제거 #####
    ready_2 = []
    for j in ready1:
        temp = j.replace("\n","")
        ready_2.append(temp)

        
    ##### 키워드,제작사 없는곳 빈부분 none 채워넣기 #####
    if (len(ready_2)==5 and ready_2[3] == ' Netflix'):
        ready_2.append(None)
        
    #####밀린 (제작사)빈칸 넣어보기#####
    #총길이len 6
    #index 3 ->제작사 
    if (len(ready_2)<=5):
        ready_2.insert(3,None)

    ##### 키워드 빈부분 none 채워넣기 #####
    if (len(ready_2)==5):
        ready_2.append(None)        
        
    ##### 제작사 앞의 빈칸제거 #####       
    if ready_2[3] != None:
        ready_2[3] = ready_2[3].strip()

        
    ##### 인덱스0에 기본키 넣기 #####   
    ready_2.insert(0,i+1)
    
    #####생성된 행 최종 리스트에 넣음 #####            
    list_all.append(ready_2)
    
    ##### 막연한 기다림을 위하여... #####
    print(i,end=" ")

print("끝!",end=" ")
##### 데이터 프레임 #####
df_netp = pd.DataFrame(list_all)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 끝! 

In [ ]:
##### df를 엑셀로 내보내기 #####

#df_netp.to_csv("detail_table.csv")

#df_main_21.to_csv("score_table_2021.csv")
#df_main_20.to_csv("score_table_2020.csv")

# 데이터베이스

In [ ]:
##### CREATE DROP #####

## 크리에이트 테이블 ##
#cursor.execute ("create table score_table(id_sc integer NOT NULL AUTO_INCREMENT PRIMARY KEY,title varchar(150),num_c integer,avg_c integer,days integer,avg_day integer,total_score integer,year varchar(5));")    
#cursor.execute("create table detail_table(  id_de integer NOT NULL PRIMARY KEY,  title varchar(150),  media varchar(20),  nation varchar(20),  prod varchar(20),  genre varchar(40),  keyword varchar(30));")

## 드랍 테이블 ##
#cursor.execute ("drop table score_table;")
#cursor.execute("drop table detail_table;")



### DB - score_table

In [11]:
###########################################################
##### "score_table"에 2020,2021년도 스코어 자료 넣기 #####

Maria_db = pymysql.connect(
    user='root', 
    passwd='1234', 
    host='127.0.0.1', 
    db='project', 
    charset='utf8'
)

cursor = Maria_db.cursor(pymysql.cursors.DictCursor)

### "드랍" / "크리에이트" ※주석처리 주의※ ###
#cursor.execute ("drop table score_table;")
#cursor.execute ("create table score_table(id_sc integer NOT NULL AUTO_INCREMENT PRIMARY KEY,title varchar(150),num_c integer,avg_c integer,days integer,avg_day integer,total_score integer,year varchar(5));")    


# 컬럼이름지정 :  df의 컬럼이름도 데이터베이스와 같아야 함
cols = "`,`".join([str(i) for i in df_main_21.columns.tolist()]) 


# 행 한줄씩 생성
for i,row in df_main_20.iterrows():
    sql = "INSERT INTO `score_table` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))
    
# 행 한줄씩 생성
for i,row in df_main_21.iterrows():
    sql = "INSERT INTO `score_table` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))    
    
Maria_db.commit()
Maria_db.close()


In [46]:
##### 결측치 채워졌는지 확인 #####
for i in range(len(list_all)):
    print(len(list_all[i]), end = " ")
    if len(list_all[i]) !=7:
        print("<",i,">")

7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 

### DB - detail_table

In [49]:
###########################################################
##### "detail_table"에 2020,2021년도 상세 자료 넣기 #####

Maria_db = pymysql.connect(
    user='root', 
    passwd='1234', 
    host='127.0.0.1', 
    db='project', 
    charset='utf8'
)
cursor = Maria_db.cursor(pymysql.cursors.DictCursor)

### "드랍" / "크리에이트" ※주석처리 주의※ ###
#cursor.execute("drop table detail_table;")
#cursor.execute("create table detail_table(  id_de integer NOT NULL PRIMARY KEY,  title varchar(150),  media varchar(20),  nation varchar(20),  prod varchar(20),  genre varchar(40),  keyword varchar(30));")

query = "insert into detail_table values(%s,%s,%s,%s,%s,%s,%s);"
for i in range(len(list_all)):
    cursor.execute(query, (list_all[i][0],
                           list_all[i][1],list_all[i][2],
                           list_all[i][3],list_all[i][4],
                           list_all[i][5],list_all[i][6]))

Maria_db.commit()
Maria_db.close()


In [ ]:
##### 데이터 타이틀 불일치 부분 update #####
'''
update score_table set title = 'Hit & Run' where title ='Hit  Run';
update score_table set title = 'Pablo Escobar, The Drug Lord' where title ='Pablo Escobar The Drug Lord';
update score_table set title = 'Locke & Key' where title ='Locke  Key';
update score_table set title = 'Love, Death & Robots' where title ='Love Death  Robots';
update score_table set title = 'Miraculous: Tales of Ladybug & Cat Noir' where title ='Miraculous: Tales of Ladybug  Cat Noir';
update score_table set title = 'Acero, Woman of Steel' where title ='Acero Woman of Steel';
update score_table set title = 'Ginny & Georgia' where title ='Ginny  Georgia';

update score_table set title = 'Tiger King: Murder, Mayhem and Madness' where title ='Tiger King: Murder Mayhem and Madness';
update score_table set title = 'Dash & Lily' where title ='Dash  Lily';
update score_table set title = 'Grizzy & the Lemmings' where title ='Grizzy  the Lemmings';
'''

# Maria DB 의 데이터를 csv로 출력 

In [3]:
######DB내  table들 csv출력#####
Maria_db = pymysql.connect(
    user='root', 
    passwd='1234', 
    host='127.0.0.1', 
    db='project', 
    charset='utf8'
)
cursor = Maria_db.cursor(pymysql.cursors.DictCursor)

cursor.execute("select * from score_table")
table_rows = cursor.fetchall()
df = pd.DataFrame(table_rows)
df.to_csv("score_table.csv")

cursor.execute("select * from detail_table")
table_rows = cursor.fetchall()
df = pd.DataFrame(table_rows)
df.to_csv("detail_table.csv")

Maria_db.commit()
Maria_db.close()
